# Comparable response of wild rodent gut microbiome to anthropogenic habitat contamination


## Abstract

Species identity is thought to dominate over environment in shaping wild rodent gut microbiota, but it remains unknown whether the responses of host gut microbiota to shared anthropogenic habitat impacts are species-specific or if the general gut microbiota response is similar across host species. Here, we compare the influence of exposure to radionuclide contamination on the gut microbiota of four wild mouse species: *Apodemus flavicollis*, *A. sylvaticus*, *A. speciosus*, *A. argenteus*. Building on the evidence that radiation impacts bank vole (*Myodes glareolus*) gut microbiota, we hypothesised that radiation exposure has general impact on rodent gut microbiota. Because we sampled (n=288) two species pairs of *Apodemus* mice that occur in sympatry in habitats affected by the Chernobyl and Fukushima nuclear accidents, these comparisons provide an opportunity for a general assessment of the effects of exposure to environmental contamination (radionuclides) on gut microbiota across host phylogeny and geographical areas. In general agreement with our hypothesis, analyses of bacterial 16S rRNA gene sequences revealed that radiation exposure alters the gut microbiota composition and structure in three out of the four species of *Apodemus* mice. The notable lack of association between the gut microbiota and soil radionuclide contamination in one mouse species from Fukushima (*A. argenteus*) likely reflects host “radiation escape” through its unique tree-dwelling lifestyle. The finding that host ecology can modulate effects of radiation exposure offers an interesting counterpoint for future analyses into effects of radiation or any other toxic exposure on host and its associated microbiota. Our data show that exposure to radionuclide contamination associates with comparable gut microbiota responses across multiple species of rodents. 


### Analysis Notebook

This notebook contains commands to process raw read data and reproduce the qiime2 analyses. This [repository](https://github.com/alavrinienko/hot-mice) provides the main data files, *i.e.* metadata, dada2 feature-tables, representative sequences, phylogenetic tree and taxonomy. It should be possible to generate the rest of the data using these files and commands. The raw sequence data for this study have been deposited in the European Nucleotide Archive (ENA) at EMBL-EBI under accession number [PRJEB44039](https://www.ebi.ac.uk/ena/browser/view/PRJEB44039). To analyse these data we have used computing resources and infrastructure provided by [the Finnish Centre for Scientific Computing (CSC)](https://www.csc.fi/csc).

### Requirements

qiime2 is required to run most of the commands described below (unless otherwise stated, the 2019.4 qiime2 version was used for analyses). To install qiime2 follow [these instructions](https://docs.qiime2.org/2019.4/install/).

## data import

In this study, we have sampled 288 mice, these include samples from 4 *Apodemus* species inhabiting areas surrounding the Chernobyl and Fukushima nuclear accidents sites. The 250PE read data were generated using the Illumina MiSeq platform. First, we need to import sequencing data to use with the qiime2 via manifest file. The manifest file will inform qiime about samples id's, full path and sequences direction (forward,reverse). See importing tutorial for more details [here](https://docs.qiime2.org/2019.4/tutorials/importing/).

Note that 16 samples were *not* analysed due to low number (<600) of reads per sample. The following samples were removed from the data set prior to import (n=16): C228, KC45, C265, C186, C309, KC20, KC5, KC22, F6, C192, F54, C259, C299, KC12, C267, C258. This should leave the total of 272 samples in this data set.

In [ ]:
!qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-path apo272_dada2_manifest.txt \
    --input-format PairedEndFastqManifestPhred33V2 \
    --output-path apo272_mani-demux_paired-end.qza

The fastq files were successfully imported into the working directory. We would want to summarize and visualize the data (e.g. imported qiime2 *qza* file) using the [qiime2 viewer](https://view.qiime2.org).

In [ ]:
!qiime demux summarize \
--i-data apo272_mani-demux_paired-end.qza \
--o-visualization apo272_mani-demux_paired-end.qzv

The generated interactive quality plots display the reads quality in defined position according to the Phred33 format. The median quality score for both forward and reverse reads was over 37, however we also observed some quality drop in the end of the reverse reads. To ensure excellent data for downstream analyses, we truncated forward reads at position 243 (the minimum shared length across all reads), and reverse reads at position 176.

Of important note, these data still contain the 515F/806R EMP primers. These, can be removed while running dada2 using trim-left parameter (the forward primer is 19 bp, and the reverse primer is 20 bp).

## denoising and the amplicon sequence variants (ASVs) picking using dada2

We used the [dada2](https://docs.qiime2.org/2019.4/plugins/available/dada2/denoise-paired/) implemented in qiime2 for denoising and the amplicon sequence variants (ASVs) picking. We adjusted parameters within the dada2 based on the information from the quality plots generated above.

In [ ]:
!qiime dada2 denoise-paired \
    --i-demultiplexed-seqs apo272_mani-demux_paired-end.qza \
    --p-trim-left-f 19 \
    --p-trim-left-r 20 \
    --p-trunc-len-f 243 \
    --p-trunc-len-r 176 \
    --p-n-threads 0 \
    --o-representative-sequences /dada2-apo272/apo272-rep-seqs-dada2.qza \
    --o-table /dada2-apo272/apo272-table-dada2.qza \
    --o-denoising-stats /dada2-apo272/apo272-denoising-stats-dada2.qza

In [ ]:
#add metadata and examine dada2 feature-table
!qiime feature-table summarize \
  --i-table dada2-apo272/apo272-table-dada2.qza \
  --o-visualization dada2-apo272/apo272-table-dada2.qzv \
  --m-sample-metadata-file apodemus_metadata191119.txt

In [ ]:
#examine representative sequences
!qiime feature-table tabulate-seqs \
 --i-data dada2-apo272/apo272-rep-seqs-dada2.qza \
 --o-visualization dada2-apo272/apo272-rep-seqs-dada2.qzv

In [ ]:
#examine denoising stats
!qiime metadata tabulate \
 --m-input-file dada2-apo272/apo272-denoising-stats-dada2.qza \
 --o-visualization dada2-apo272/apo272-denoising-stats-dada2.qzv

## metadata

Note that all the metadata associated with the samples in this study can be found [here](https://github.com/alavrinienko/hot-mice). Note that the file 'metadata-apo272.txt' is the most complete version of the mice metadata, which in this document are sometimes referred to as 'apodemus_metadata191119.txt' or 'apodemus_metadata221119.txt' or 'apodemus_metadata181219.txt' or 'apodemus_metadata181219_rev1ME.txt', etc. The metadata associated with the *comparative analysis: voles and mice data sets* section (see below) can be found in the 'metadata-vole137-apo272.txt' file.

## phylogenetics

Construct a phylogenetic tree using FastTree, and use the longest branch to root the tree (‘midrooting’).

In [ ]:
#first, align the representative sequences
!qiime alignment mafft \
  --i-sequences dada2-apo272/apo272-rep-seqs-dada2.qza \
  --o-alignment dada2-apo272/aligned-apo272-rep-seqs-dada2.qza

#mask phyloginetically unimportant sites
!qiime alignment mask \
  --i-alignment dada2-apo272/aligned-apo272-rep-seqs-dada2.qza \
  --o-masked-alignment dada2-apo272/masked-aligned-apo272-rep-seqs-dada2.qza

#create a tree
!qiime phylogeny fasttree \
  --i-alignment dada2-apo272/masked-aligned-apo272-rep-seqs-dada2.qza \
  --o-tree dada2-apo272/unrooted-apo272-tree-dada2.qza

#root generated tree at midpoint
!qiime phylogeny midpoint-root \
  --i-tree dada2-apo272/unrooted-apo272-tree-dada2.qza \
  --o-rooted-tree dada2-apo272/rooted-apo272-tree-dada2.qza

## feature-table filtering

Remove low-abundance features from the feature-table (features with the frequency <10 across all samples). It is important to filter out potential errors/artifacts that were retained during the denoising step, thus this needs to be done before the downstream core metrics analysis. Such features filtering strategy (with the cut off of 10) was adopted from the --p-min-reads default parameters in the [qiime2 deblur workflow](https://docs.qiime2.org/2019.4/plugins/available/deblur/denoise-16S/), which retains only features appearing at least min_reads 10 times across all samples in the resulting feature-table by default.

In [ ]:
#filter features with frequency less than 10
!qiime feature-table filter-features \
  --i-table dada2-apo272/apo272-table-dada2.qza \
  --p-min-frequency 10 \
  --o-filtered-table dada2-apo272/apo272-table-dada2-filtered10.qza

In [ ]:
#summarize filtered feature-table
!qiime feature-table summarize \
  --i-table dada2-apo272/apo272-table-dada2-filtered10.qza \
  --o-visualization dada2-apo272/apo272-table-dada2-filtered10.qzv \
  --m-sample-metadata-file apodemus_metadata191119.txt

## rarefaction

Examine sequencing depth across samples that were processed using dada2 (and filtered), and choose the even sampling depth for rarefaction. Use the median frequency across samples in the dataset (38599). This limit will be used to generate alpha rarefaction curves to examine sequencing coverage.

In [ ]:
#make alpha-rarefaction plot and estimate community coverage
!qiime diversity alpha-rarefaction \
 --i-table dada2-apo272/apo272-table-dada2-filtered10.qza \
 --i-phylogeny dada2-apo272/rooted-apo272-tree-dada2.qza \
 --p-max-depth 38599 \
 --p-metrics faith_pd \
 --p-metrics shannon \
 --p-metrics observed_otus \
 --p-metrics chao1 \
 --m-metadata-file apodemus_metadata191119.txt \
 --o-visualization dada2-apo272/alpha-rare-apo272-dada2-filtered10.qzv

Based on the alpha-rarefaction and the table generated by dada2, we would rarefy the data set at the 10553 (minimum sample frequency) reads per sample. This is because the rarefaction curve level off at this value. With this even sampling depth we likely sample most of the community, while analysing all the samples.

In [ ]:
#run core-metrics pipeline at a given sequencing depth, using dada2 filtered10 data
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny dada2-apo272/rooted-apo272-tree-dada2.qza \
  --i-table dada2-apo272/apo272-table-dada2-filtered10.qza \
  --p-sampling-depth 10553 \
  --m-metadata-file apodemus_metadata191119.txt \
  --output-dir dada2-apo272/core-metrics-apo272-dada2-filtered10-results/

## taxonomic analysis

Upload the GreenGenes 'reference data base'. This is a pre-trained Naive Bayes classifier for the use with the q2-feature-classifier plugin. This classifier was trained on the Greengenes 13_8 99% OTUs, where the sequences have been trimmed to only include 250 bases from the region of the 16S that was sequenced in this study (the V4 region, bound by the 515F/806R EMP primers). See more information of the pre-trained classifiers [here](https://docs.qiime2.org/2019.4/data-resources/) (MD5:3afcc86150423263b3a7d983789ad0a3).

In [ ]:
!qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads dada2-apo272/apo272-rep-seqs-dada2.qza \
  --o-classification dada2-apo272/apo272-dada2-taxonomy.qza

!qiime metadata tabulate \
  --m-input-file dada2-apo272/apo272-dada2-taxonomy.qza \
  --o-visualization dada2-apo272/apo272-dada2-taxonomy.qzv

Examine the microbiota taxonomic composition with the interactive bar plots. Rarefaction procedure in the core-metric pipeline above, samples reads at random. Thus, for the complete match in the data comparison, examine the rarefied feature-table found in [this](https://github.com/alavrinienko/hot-mice) repository at GitHub.

In [ ]:
#create the taxa bar plots using the 'filtered10' and rarefied at 10553 reads/sample feature-table
!qiime taxa barplot \
  --i-table dada2-apo272/core-metrics-apo272-dada2-filtered10-results/rarefied_table.qza \
  --i-taxonomy dada2-apo272/apo272-dada2-taxonomy.qza \
  --m-metadata-file apodemus_metadata191119.txt \
  --o-visualization dada2-apo272/apo272-dada2-filtered10-even10553-taxa-bar-plots.qzv


## subsample the feature-table
Use the rarefied feature-table to subsample the data set into subsets for analyses according to host species and locality (or some other combination, see below).

In [ ]:
!qiime feature-table filter-samples \
  --i-table dada2-apo272/core-metrics-apo272-dada2-filtered10-results/rarefied_table.qza \
  --m-metadata-file apodemus_metadata191119.txt \
  --p-where "Study='Chernobyl'" \
  --o-filtered-table dada2-apo272/rarefied_table_chernobyl142.qza

In [ ]:
!qiime feature-table filter-samples \
  --i-table dada2-apo272/core-metrics-apo272-dada2-filtered10-results/rarefied_table.qza \
  --m-metadata-file apodemus_metadata191119.txt \
  --p-where "Study='Fukushima'" \
  --o-filtered-table dada2-apo272/rarefied_table_fukushima130.qza

In [ ]:
#summarize filtered 'Chernobyl-only' feature-table
!qiime feature-table summarize \
  --i-table dada2-apo272/rarefied_table_chernobyl142.qza \
  --o-visualization dada2-apo272/rarefied_table_chernobyl142.qzv \
  --m-sample-metadata-file apodemus_metadata191119.txt

#summarize filtered 'Fukushima-only'feature-table
!qiime feature-table summarize \
  --i-table dada2-apo272/rarefied_table_fukushima130.qza \
  --o-visualization dada2-apo272/rarefied_table_fukushima130.qzv \
  --m-sample-metadata-file apodemus_metadata191119.txt

Run the qiime2 core-metrics-phylogenetic pipeline on the filtered 'chernobyl142.qza' and 'fukushima130.qza' feature-tables. Note that here (and during similar steps below..) we still provided the rarefaction depth of 10553, yet the data were already rarefied and each sample has even 10553 reads, so no changes to the data can happen due to this (sanity checked - OK).

In [ ]:
#run core-metrics pipeline at a given sequencing depth, using dada2 filtered10 CHERNOBYL data
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny dada2-apo272/rooted-apo272-tree-dada2.qza \
  --i-table dada2-apo272/rarefied_table_chernobyl142.qza \
  --p-sampling-depth 10553 \
  --m-metadata-file apodemus_metadata191119.txt \
  --output-dir dada2-apo272/core-metrics-chernobyl142-dada2-filtered10-results/

In [ ]:
#run core-metrics pipeline at a given sequencing depth, using dada2 filtered10 FUKUSHIMA data
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny dada2-apo272/rooted-apo272-tree-dada2.qza \
  --i-table dada2-apo272/rarefied_table_fukushima130.qza \
  --p-sampling-depth 10553 \
  --m-metadata-file apodemus_metadata191119.txt \
  --output-dir dada2-apo272/core-metrics-fukushima130-dada2-filtered10-results/

Use the filtered 'chernobyl142.qza' feature-table (CHERNOBYL data only) to subsample the data set into subsets for analyses according to host species - to include either samples from *Apodemus sylvaticus* OR *Apodemus flavicollis* only.

In [ ]:
#keep Apodemus sylvaticus samples only
!qiime feature-table filter-samples \
  --i-table dada2-apo272/rarefied_table_chernobyl142.qza \
  --m-metadata-file apodemus_metadata221119.txt \
  --p-where "ApodemusSpecies='ApodemusSylvaticus'" \
  --o-filtered-table dada2-apo272/rarefied_table_chernobyl27_sylvaticus.qza

In [ ]:
#summarize filtered feature-table
!qiime feature-table summarize \
  --i-table dada2-apo272/rarefied_table_chernobyl27_sylvaticus.qza \
  --o-visualization dada2-apo272/rarefied_table_chernobyl27_sylvaticus.qzv \
  --m-sample-metadata-file apodemus_metadata221119.txt

In [ ]:
#keep Apodemus flavicollis samples only
!qiime feature-table filter-samples \
  --i-table dada2-apo272/rarefied_table_chernobyl142.qza \
  --m-metadata-file apodemus_metadata221119.txt \
  --p-where "ApodemusSpecies='ApodemusFlavicollis'" \
  --o-filtered-table dada2-apo272/rarefied_table_chernobyl115_flavicollis.qza

In [ ]:
#summarize filtered feature-table
!qiime feature-table summarize \
  --i-table dada2-apo272/rarefied_table_chernobyl115_flavicollis.qza \
  --o-visualization dada2-apo272/rarefied_table_chernobyl115_flavicollis.qzv \
  --m-sample-metadata-file apodemus_metadata221119.txt

In [ ]:
#run core-metrics pipeline at a given sequencing depth, using dada2 filtered10 A.sylvaticus data
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny dada2-apo272/rooted-apo272-tree-dada2.qza \
  --i-table dada2-apo272/rarefied_table_chernobyl27_sylvaticus.qza \
  --p-sampling-depth 10553 \
  --m-metadata-file apodemus_metadata221119.txt \
  --output-dir dada2-apo272/core-metrics-chernobyl27-sylvaticus-dada2-filtered10-results/

In [ ]:
#run core-metrics pipeline at a given sequencing depth, using dada2 filtered10 A.flavicollis data
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny dada2-apo272/rooted-apo272-tree-dada2.qza \
  --i-table dada2-apo272/rarefied_table_chernobyl115_flavicollis.qza \
  --p-sampling-depth 10553 \
  --m-metadata-file apodemus_metadata221119.txt \
  --output-dir dada2-apo272/core-metrics-chernobyl115-flavicollis-dada2-filtered10-results/

Use the filtered 'fukushima130.qza' feature-table (FUKUSHIMA data only) to subsample the data set into subsets for analyses according to host species - to include either samples from *Apodemus argenteus* OR *Apodemus speciosus* only.

In [ ]:
#keep Apodemus argenteus (small) samples only
!qiime feature-table filter-samples \
  --i-table dada2-apo272/rarefied_table_fukushima130.qza \
  --m-metadata-file apodemus_metadata221119.txt \
  --p-where "ApodemusSpecies='ApodemusArgenteusSmall'" \
  --o-filtered-table dada2-apo272/rarefied_table_fukushima53_argenteus_small.qza

In [ ]:
#summarize filtered feature-table
!qiime feature-table summarize \
  --i-table dada2-apo272/rarefied_table_fukushima53_argenteus_small.qza \
  --o-visualization dada2-apo272/rarefied_table_fukushima53_argenteus_small.qzv \
  --m-sample-metadata-file apodemus_metadata221119.txt

In [ ]:
#keep Apodemus speciosus (large) samples only
!qiime feature-table filter-samples \
  --i-table dada2-apo272/rarefied_table_fukushima130.qza \
  --m-metadata-file apodemus_metadata221119.txt \
  --p-where "ApodemusSpecies='ApodemusSpeciosusLarge'" \
  --o-filtered-table dada2-apo272/rarefied_table_fukushima77_speciosus_large.qza

In [ ]:
#summarize filtered feature-table
!qiime feature-table summarize \
  --i-table dada2-apo272/rarefied_table_fukushima77_speciosus_large.qza \
  --o-visualization dada2-apo272/rarefied_table_fukushima77_speciosus_large.qzv \
  --m-sample-metadata-file apodemus_metadata221119.txt

In [ ]:
#run core-metrics pipeline at a given sequencing depth, using dada2 filtered10 A.argenteus data
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny dada2-apo272/rooted-apo272-tree-dada2.qza \
  --i-table dada2-apo272/rarefied_table_fukushima53_argenteus_small.qza \
  --p-sampling-depth 10553 \
  --m-metadata-file apodemus_metadata221119.txt \
  --output-dir dada2-apo272/core-metrics-fukushima53-argenteus-small-dada2-filtered10-results/

In [ ]:
#run core-metrics pipeline at a given sequencing depth, using dada2 filtered10 A.speciosus data
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny dada2-apo272/rooted-apo272-tree-dada2.qza \
  --i-table dada2-apo272/rarefied_table_fukushima77_speciosus_large.qza \
  --p-sampling-depth 10553 \
  --m-metadata-file apodemus_metadata221119.txt \
  --output-dir dada2-apo272/core-metrics-fukushima77-speciosus-large-dada2-filtered10-results/

## taxa relative frequency tables
Generate the tables with relative abundance data at several taxanomical levels (L2 phylum, L3 class, L4 order, L5 family, L6 genus, L7 species). These are based on the 'filtered10' and rarefied at 10553 reads/sample feature-table (used for plotting and stats).

In [ ]:
!qiime taxa collapse \
--i-table dada2-apo272/core-metrics-apo272-dada2-filtered10-results/rarefied_table.qza \
--i-taxonomy dada2-apo272/apo272-dada2-taxonomy.qza \
--p-level 2 \
--output-dir dada2-apo272/taxa-rel-abund/L2

In [ ]:
!qiime feature-table relative-frequency \
--i-table dada2-apo272/taxa-rel-abund/L2/collapsed_table.qza \
--output-dir dada2-apo272/taxa-rel-abund/L2/relative-L2

In [ ]:
!qiime taxa collapse \
--i-table dada2-apo272/core-metrics-apo272-dada2-filtered10-results/rarefied_table.qza \
--i-taxonomy dada2-apo272/apo272-dada2-taxonomy.qza \
--p-level 3 \
--output-dir dada2-apo272/taxa-rel-abund/L3

In [ ]:
!qiime feature-table relative-frequency \
--i-table dada2-apo272/taxa-rel-abund/L3/collapsed_table.qza \
--output-dir dada2-apo272/taxa-rel-abund/L3/relative-L3

In [ ]:
!qiime taxa collapse \
--i-table dada2-apo272/core-metrics-apo272-dada2-filtered10-results/rarefied_table.qza \
--i-taxonomy dada2-apo272/apo272-dada2-taxonomy.qza \
--p-level 4 \
--output-dir dada2-apo272/taxa-rel-abund/L4

In [ ]:
!qiime feature-table relative-frequency \
--i-table dada2-apo272/taxa-rel-abund/L4/collapsed_table.qza \
--output-dir dada2-apo272/taxa-rel-abund/L4/relative-L4

In [ ]:
!qiime taxa collapse \
--i-table dada2-apo272/core-metrics-apo272-dada2-filtered10-results/rarefied_table.qza \
--i-taxonomy dada2-apo272/apo272-dada2-taxonomy.qza \
--p-level 5 \
--output-dir dada2-apo272/taxa-rel-abund/L5

In [ ]:
!qiime feature-table relative-frequency \
--i-table dada2-apo272/taxa-rel-abund/L5/collapsed_table.qza \
--output-dir dada2-apo272/taxa-rel-abund/L5/relative-L5

In [ ]:
!qiime taxa collapse \
--i-table dada2-apo272/core-metrics-apo272-dada2-filtered10-results/rarefied_table.qza \
--i-taxonomy dada2-apo272/apo272-dada2-taxonomy.qza \
--p-level 6 \
--output-dir dada2-apo272/taxa-rel-abund/L6

In [ ]:
!qiime feature-table relative-frequency \
--i-table dada2-apo272/taxa-rel-abund/L6/collapsed_table.qza \
--output-dir dada2-apo272/taxa-rel-abund/L6/relative-L6

In [ ]:
#open relative-frequency_table.qza with any zip archive app and export biom table,
#then, run the following:
!module load qiime/1.9.1 #this is to load qiime1 env as it contains biom package
!biom convert -i dada2-apo272/taxa-rel-abund/L2-feature-table.biom -o dada2-apo272/taxa-rel-abund/L2-feature-table.tsv tsv
!biom convert -i dada2-apo272/taxa-rel-abund/L3-feature-table.biom -o dada2-apo272/taxa-rel-abund/L3-feature-table.tsv tsv
!biom convert -i dada2-apo272/taxa-rel-abund/L4-feature-table.biom -o dada2-apo272/taxa-rel-abund/L4-feature-table.tsv tsv
!biom convert -i dada2-apo272/taxa-rel-abund/L5-feature-table.biom -o dada2-apo272/taxa-rel-abund/L5-feature-table.tsv tsv
!biom convert -i dada2-apo272/taxa-rel-abund/L6-feature-table.biom -o dada2-apo272/taxa-rel-abund/L6-feature-table.tsv tsv

## beta diversity

Identify factors that are responsible for the patterns observed in the beta diversity ordinations (including numerical data categories). For this, we used the adonis function in the vegan R package implemented in qiime2 (qiime2 actually uses the R-vegan under the hood; outputs of the adonis vegan package function are consistent between R-native vegan and qiime2-vegan implementation - sanity checked - OK). The input distance files for the adonis tests derive from the core-diversity-phylogenetic pipeline (described above). Examine the output files and archive these data for the manuscript.
Note that all four beta diversity metrics were used throughout, e.g. the Bray-Curtis, Jaccard, unweigthed and weighted UniFrac metrics.

**test 1**: Examine effects of the country of origin (Ukraine or Japan) and host species identiy using the following formula: "Study+ApodemusSpecies".

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-apo272-dada2-filtered10-results/bray_curtis_distance_matrix.qza \
--m-metadata-file apodemus_metadata101219.txt \
--p-formula "Study+ApodemusSpecies" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-bray-apo272-study-apodemussp.qzv

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-apo272-dada2-filtered10-results/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata101219.txt \
--p-formula "Study+ApodemusSpecies" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-unwUF-apo272-study-apodemussp.qzv

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-apo272-dada2-filtered10-results/weighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata101219.txt \
--p-formula "Study+ApodemusSpecies" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-weightUF-apo272-study-apodemussp.qzv

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-apo272-dada2-filtered10-results/jaccard_distance_matrix.qza \
--m-metadata-file apodemus_metadata101219.txt \
--p-formula "Study+ApodemusSpecies" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-jaccard-apo272-study-apodemussp.qzv

**test 2**: Use univariate models including only host species in either study site (Chernobyl or Fukushima; see below) to compare the variance explained by host species identity. Chernobyl data only.

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl142-dada2-filtered10-results/bray_curtis_distance_matrix.qza \
--m-metadata-file apodemus_metadata101219.txt \
--p-formula "ApodemusSpecies" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-bray-chernobyl142-apodemussp.qzv

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl142-dada2-filtered10-results/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata101219.txt \
--p-formula "ApodemusSpecies" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-unwUF-chernobyl142-apodemussp.qzv

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl142-dada2-filtered10-results/weighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata101219.txt \
--p-formula "ApodemusSpecies" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-weightUF-chernobyl142-apodemussp.qzv

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl142-dada2-filtered10-results/jaccard_distance_matrix.qza \
--m-metadata-file apodemus_metadata101219.txt \
--p-formula "ApodemusSpecies" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-jaccard-chernobyl142-apodemussp.qzv

**test 2.1**: Fukushima data only.

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-fukushima130-dada2-filtered10-results/bray_curtis_distance_matrix.qza \
--m-metadata-file apodemus_metadata101219.txt \
--p-formula "ApodemusSpecies" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-bray-fukushima130-apodemussp.qzv

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-fukushima130-dada2-filtered10-results/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata101219.txt \
--p-formula "ApodemusSpecies" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-unwUF-fukushima130-apodemussp.qzv

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-fukushima130-dada2-filtered10-results/weighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata101219.txt \
--p-formula "ApodemusSpecies" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-weightUF-fukushima130-apodemussp.qzv

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-fukushima130-dada2-filtered10-results/jaccard_distance_matrix.qza \
--m-metadata-file apodemus_metadata101219.txt \
--p-formula "ApodemusSpecies" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-jaccard-fukushima130-apodemussp.qzv

**test 3**: Construct full multivariate models to examine potential drivers of the variation in the gut microbiome of each *Apodemus* host species studied (in both, Chernobyl and Fukushima): *Apodemus flavicollis* ('apo115'), *Apodemus sylvaticus* ('apo27'), *Apodemus argenteus* ('apo53'), *Apodemus speciosus* ('apo77'). Here we test for: **(1)** treatment; **(2)** sex; **(3)** body mass; **(4)** head width; **(5)** host state; **(6)** radiation dose rate (total exposure estimates). Fit models with the variables in the following order: "Treatment+TotalDRmGydCs134forFUKU+HostSex+HostState+HeadWidthmm+BodyMassg". Note that we have also run the tests with the body condition index estimates instead of the 'BodyMass+Headwidth', as they are inclusive (yet, this makes no difference to the overall results so this part is not reported further). We also test for differences in dispersion between the treatment groups using the permdisp function.

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl115-flavicollis-dada2-filtered10-results/bray_curtis_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HostState+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-bray-chernobyl115-flav-Tr-Tot-Sx-Hst-Hw-Bm.qzv

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl115-flavicollis-dada2-filtered10-results/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HostState+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-unwUF-chernobyl115-flav-Tr-Tot-Sx-Hst-Hw-Bm.qzv

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl115-flavicollis-dada2-filtered10-results/weighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HostState+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-weightUF-chernobyl115-flav-Tr-Tot-Sx-Hst-Hw-Bm.qzv

In [ ]:
!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl115-flavicollis-dada2-filtered10-results/jaccard_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HostState+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-jaccard-chernobyl115-flav-Tr-Tot-Sx-Hst-Hw-Bm.qzv

**test 3.1**: Also, run permdisp to see if the differences in permanova adonis tests for *Apodemus flavicollis* ('apo115') are driven by the true difference in means between groups or their dispersion.

In [ ]:
!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl115-flavicollis-dada2-filtered10-results/bray_curtis_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-bray-chernobyl115-flav-treatment.qzv

In [ ]:
!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl115-flavicollis-dada2-filtered10-results/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-unwUF-chernobyl115-flav-treatment.qzv

In [ ]:
!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl115-flavicollis-dada2-filtered10-results/weighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-weightUF-chernobyl115-flav-treatment.qzv

In [ ]:
!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl115-flavicollis-dada2-filtered10-results/jaccard_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-jaccard-chernobyl115-flav-treatment.qzv

Repeat **tests 3 and 3.1** for the *Apodemus sylvaticus* ('apo27') data set from Chernobyl.

In [ ]:
#test 3: adonis tests for Apodemus sylvaticus (apo27)

!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl27-sylvaticus-dada2-filtered10-results/bray_curtis_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HostState+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-bray-chernobyl27-sylv-Tr-Tot-Sx-Hst-Hw-Bm.qzv

!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl27-sylvaticus-dada2-filtered10-results/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HostState+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-unwUF-chernobyl27-sylv-Tr-Tot-Sx-Hst-Hw-Bm.qzv

!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl27-sylvaticus-dada2-filtered10-results/weighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HostState+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-weightUF-chernobyl27-sylv-Tr-Tot-Sx-Hst-Hw-Bm.qzv

!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl27-sylvaticus-dada2-filtered10-results/jaccard_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HostState+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-jaccard-chernobyl27-sylv-Tr-Tot-Sx-Hst-Hw-Bm.qzv

In [ ]:
#test 3.1: permdisp tests for Apodemus sylvaticus (apo27)

!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl27-sylvaticus-dada2-filtered10-results/bray_curtis_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-bray-chernobyl27-sylv-treatment.qzv

!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl27-sylvaticus-dada2-filtered10-results/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-unwUF-chernobyl27-sylv-treatment.qzv

!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl27-sylvaticus-dada2-filtered10-results/weighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-weightUF-chernobyl27-sylv-treatment.qzv

!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-chernobyl27-sylvaticus-dada2-filtered10-results/jaccard_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-jaccard-chernobyl27-sylv-treatment.qzv

Repeat **tests 3 and 3.1** for the *Apodemus argenteus* ('apo53') data set from Fukushima.

In [ ]:
#test 3: adonis tests for Apodemus argenteus (apo53)

!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-fukushima53-argenteus-small-dada2-filtered10-results/bray_curtis_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HostState+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-bray-fukushima53-argent-Tr-Tot-Sx-Hst-Hw-Bm.qzv

!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-fukushima53-argenteus-small-dada2-filtered10-results/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HostState+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-unwUF-fukushima53-argent-Tr-Tot-Sx-Hst-Hw-Bm.qzv

!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-fukushima53-argenteus-small-dada2-filtered10-results/weighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HostState+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-weightUF-fukushima53-argent-Tr-Tot-Sx-Hst-Hw-Bm.qzv

!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-fukushima53-argenteus-small-dada2-filtered10-results/jaccard_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HostState+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-jaccard-fukushima53-argent-Tr-Tot-Sx-Hst-Hw-Bm.qzv

In [ ]:
#test 3.1: permdisp tests for Apodemus argenteus (apo53)

!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-fukushima53-argenteus-small-dada2-filtered10-results/bray_curtis_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-bray-fukushima53-argent-treatment.qzv

!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-fukushima53-argenteus-small-dada2-filtered10-results/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-unwUF-fukushima53-argent-treatment.qzv

!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-fukushima53-argenteus-small-dada2-filtered10-results/weighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-weightUF-fukushima53-argent-treatment.qzv

!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-fukushima53-argenteus-small-dada2-filtered10-results/jaccard_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-jaccard-fukushima53-argent-treatment.qzv

Repeat **tests 3 and 3.1** for the *Apodemus speciosus* ('apo77') dataset from Fukushima. Note, that the formula here have no 'HostState' variable as there is only one level under this term (e.g. all of the sampled individuals were adults).

In [ ]:
#test 3: adonis tests for Apodemus speciosus (apo77)

!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-fukushima77-speciosus-large-dada2-filtered10-results/bray_curtis_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-bray-fukushima77-specios-Tr-Tot-Sx-Hw-Bm.qzv

!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-fukushima77-speciosus-large-dada2-filtered10-results/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-unwUF-fukushima77-specios-Tr-Tot-Sx-Hw-Bm.qzv

!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-fukushima77-speciosus-large-dada2-filtered10-results/weighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-weightUF-fukushima77-specios-Tr-Tot-Sx-Hw-Bm.qzv

!qiime diversity adonis \
--i-distance-matrix dada2-apo272/core-metrics-fukushima77-speciosus-large-dada2-filtered10-results/jaccard_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--p-formula "Treatment+TotalDRmGydCs134forFUKU+HostSex+HeadWidthmm+BodyMassg" \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-adonis-jaccard-fukushima77-specios-Tr-Tot-Sx-Hw-Bm.qzv

In [ ]:
#test 3.1: permdisp tests for Apodemus speciosus (apo77)

!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-fukushima77-speciosus-large-dada2-filtered10-results/bray_curtis_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-bray-fukushima77-specios-treatment.qzv

!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-fukushima77-speciosus-large-dada2-filtered10-results/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-unwUF-fukushima77-specios-treatment.qzv

!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-fukushima77-speciosus-large-dada2-filtered10-results/weighted_unifrac_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-weightUF-fukushima77-specios-treatment.qzv

!qiime diversity beta-group-significance \
--i-distance-matrix dada2-apo272/core-metrics-fukushima77-speciosus-large-dada2-filtered10-results/jaccard_distance_matrix.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column Treatment \
--p-method 'permdisp' \
--o-visualization dada2-apo272/permanova-bdiv-tests/q2-permdisp-jaccard-fukushima77-specios-treatment.qzv

## classify samples using Random Forest classifier
Examine the Random Forest predictions for the country of origin: Ukraine vs. Japan; and *Apodemus* species based on the gut microbiome community composition. See some useful discussion along the topic [here](https://joss.theoj.org/papers/10.21105/joss.00934), [here](https://docs.qiime2.org/2019.4/plugins/available/sample-classifier/classify-samples/) and [here](https://docs.qiime2.org/2019.4/tutorials/sample-classifier/).

In [ ]:
!qiime sample-classifier classify-samples \
  --i-table dada2-apo272/core-metrics-apo272-dada2-filtered10-results/rarefied_table.qza \
  --m-metadata-file apodemus_metadata181219.txt \
  --m-metadata-column Study \
  --p-optimize-feature-selection \
  --p-parameter-tuning \
  --p-estimator RandomForestClassifier \
  --p-n-estimators 10000 \
  --output-dir dada2-apo272/random-forest/rf-apo272-study

!qiime sample-classifier classify-samples \
  --i-table dada2-apo272/core-metrics-apo272-dada2-filtered10-results/rarefied_table.qza \
  --m-metadata-file apodemus_metadata181219.txt \
  --m-metadata-column ApodemusSpecies \
  --p-optimize-feature-selection \
  --p-parameter-tuning \
  --p-estimator RandomForestClassifier \
  --p-n-estimators 10000 \
  --output-dir dada2-apo272/random-forest/rf-apo272-apodemussp

Examine the most important features (e.g. ASVs) for classification of the samples from the respective *Apodemus* host species.

In [ ]:
!qiime feature-table filter-features \
  --i-table dada2-apo272/core-metrics-apo272-dada2-filtered10-results/rarefied_table.qza \
  --m-metadata-file dada2-apo272/random-forest/rf-apo272-apodemussp/feature_importance.qza \
  --o-filtered-table dada2-apo272/random-forest/rf-apo272-apodemussp/important-1226feature-rarefied-apo272-table.qza

In [ ]:
#summarize filtered feature-table
!qiime feature-table summarize \
  --i-table dada2-apo272/random-forest/rf-apo272-apodemussp/important-1226feature-rarefied-apo272-table.qza \
  --o-visualization dada2-apo272/random-forest/rf-apo272-apodemussp/important-1226feature-rarefied-apo272-table.qzv \
  --m-sample-metadata-file apodemus_metadata181219.txt

Use [this](https://docs.qiime2.org/2019.4/plugins/available/sample-classifier/heatmap/) qiime2 plugin to make a heatmap with the top20 ASVs that discriminate *Apodemus* species the most (sanity-checked - OK). Note that the feature-table was normalised by adding a psuedocount of 1 and then taking the log10 of the table to aid plotting. Also, average clustering was performed using Bray-Curtis dissimilarity metric.

In [ ]:
!qiime sample-classifier heatmap \
  --i-table dada2-apo272/random-forest/rf-apo272-apodemussp/important-1226feature-rarefied-apo272-table.qza \
  --i-importance dada2-apo272/random-forest/rf-apo272-apodemussp/feature_importance.qza \
  --m-metadata-file apodemus_metadata181219.txt \
  --m-metadata-column ApodemusSpecies \
  --p-group-samples \
  --p-color-scheme 'viridis' \
  --p-feature-count 20 \
  --p-cluster 'both' \
  --o-filtered-table dada2-apo272/random-forest/rf-apo272-apodemussp/both-important-to-20feature-rarefied-apo272-table.qza \
  --o-heatmap dada2-apo272/random-forest/rf-apo272-apodemussp/both-important-top20feature-heatmap.qzv

## diversity of the *S24-7* family
Examine diversity and distribution of the ASVs assigned to the *S24-7* family in *Apodemus flavicollis*. 

The scripts below were used to examine whether diversity of the '*S24-7* only' feature-table is identical in the qiime2 based and R-phyloseq based estimates - yes the two approaches provide identical counts/results - sanity-checked - OK.

In [ ]:
!qiime taxa filter-table \
  --i-table dada2-apo272/core-metrics-chernobyl115-flavicollis-dada2-filtered10-results/rarefied_table.qza \
  --i-taxonomy dada2-apo272/apo272-dada2-taxonomy.qza \
  --p-include f__S24-7 \
  --o-filtered-table dada2-apo272/diversity-s247/apo115-flav-table-with-S247-only.qza

In [ ]:
#summarize filtered feature-table - sanity checked OK
!qiime feature-table summarize \
  --i-table dada2-apo272/diversity-s247/apo115-flav-table-with-S247-only.qza \
  --o-visualization dada2-apo272/diversity-s247/apo115-flav-table-with-S247-only.qzv \
  --m-sample-metadata-file apodemus_metadata181219.txt

In [ ]:
#compute non-phylogenetic alpha-diversity metrics - note that this was done using the qiime2-2020.8 (version change)
!qiime diversity alpha \
--i-table dada2-apo272/diversity-s247/apo115-flav-table-with-S247-only.qza \
--p-metric 'observed_features' \
--o-alpha-diversity dada2-apo272/diversity-s247/apo115-flav-S247-only-adiv.qza \

## comparative analysis: voles and mice data sets

Import the 16S data for 137 bank voles (*Myodes glareolus*) hosts from [Lavrinienko et al. 2018](https://www.nature.com/articles/s41396-018-0214-x) into qiime2, and process these read data similarly as the mice data described above, to ensure the data sets are comparable. The raw sequence data for this study are available for download from the European Nucleotide Archive (ENA) at EMBL-EBI under accession number [ERP104266](https://www.ebi.ac.uk/ena/browser/view/PRJEB22579) and [QIITA](https://qiita.ucsd.edu/) study ID 11360.

Of important note, these data are free of adapters and primers. These, including any other sequencing constucts were removed as a part of the sequencing facility service, so no additional trimming from the left needed. This was run with the qiime2 version change to 2020.2.

In [ ]:
!qiime tools import \
--type 'SampleData[PairedEndSequencesWithQuality]' \
--input-path manifest-vole137-apo272.txt \
--input-format PairedEndFastqManifestPhred33V2 \
--output-path demux-vole137.qza

In [ ]:
!qiime demux summarize \
--i-data demux-vole137.qza \
--o-visualization demux-vole137.qzv

In [ ]:
!qiime dada2 denoise-paired \
--i-demultiplexed-seqs demux-vole137.qza \
--p-trunc-len-f 250 \
--p-trunc-len-r 166 \
--p-trim-left-f 0 \
--p-trim-left-r 0 \
--p-n-threads 0 \
--o-table dada2-table-vole137.qza \
--o-representative-sequences dada2-rep-seqs-vole137.qza \
--o-denoising-stats dada2-stats-vole137.qza \

#examine dada2 feature-table
!qiime feature-table summarize \
  --i-table dada2-table-vole137.qza \
  --o-visualization dada2-table-vole137.qzv \
  
#examine representative sequences
!qiime feature-table tabulate-seqs \
 --i-data dada2-rep-seqs-vole137.qza \
 --o-visualization dada2-rep-seqs-vole137.qzv
 
#examine denoising stats
!qiime metadata tabulate \
 --m-input-file dada2-stats-vole137.qza \
 --o-visualization dada2-stats-vole137.qzv

Filter low-abundance features (with the frequency of less than 10)

In [ ]:
!qiime feature-table filter-features \
  --i-table dada2-table-vole137.qza \
  --p-min-frequency 10 \
  --o-filtered-table dada2-table-filt10-vole137.qza
  
#summarize the filtered10 feature table
!qiime feature-table summarize \
  --i-table dada2-table-filt10-vole137.qza \
  --o-visualization dada2-table-filt10-vole137.qzv \

In [ ]:
#make alpha-rarefaction plot and estimate community coverage
#median value was taken as a max-depth parameter
!qiime diversity alpha-rarefaction \
 --i-table dada2-table-filt10-vole137.qza \
 --p-max-depth 31066 \
 --p-metrics shannon \
 --p-metrics observed_otus \
 --m-metadata-file metadata-vole137-apo272.txt \
 --o-visualization alpha-rare-dada2-table-filt10-vole137.qzv

In [ ]:
#the same 10553 reads/sample rarefaction depth was applied for comparison
!qiime diversity core-metrics \
  --i-table dada2-table-filt10-vole137.qza \
  --p-sampling-depth 10553 \
  --m-metadata-file metadata-vole137-apo272.txt \
  --output-dir core-metrics-filt10-vole137-results/

Merge this filtered and rarefied vole feature-table with the feature-table with all the 272 mice samples ('filtered10', 'rarefied' dada2 feature-table described above). Also, merge their associated representative sequences. This is possible because the feature (ASVs) hashed ids generated in each run of dada2 are directly comparable (in this case, the feature ids are the MD5 hash of the sequence defining the feature). **Note** that the 'apo272-original-files' directory contains the original mice data files generated using scripts described above. Note that the '-vole137' was added to the 'core-metrics-filt10-vole137-results/rarefied_table.qza' file name for clarity and unique files name list (e.g. resulted in 'rarefied_table-vole137.qza').

In [ ]:
#merge tables
!qiime feature-table merge \
  --i-tables core-metrics-filt10-vole137-results/rarefied_table-vole137.qza \
  --i-tables apo272-original-files/rarefied_table.qza \
  --o-merged-table merged-dada2-filt10-raref-table-vole137-apo272.qza

#merge rep-seqs
!qiime feature-table merge-seqs \
  --i-data dada2-rep-seqs-vole137.qza \
  --i-data apo272-original-files/apo272-rep-seqs-dada2.qza \
  --o-merged-data merged-dada2-rep-seqs-vole137-apo272.qza


#examine outputs
#examine merged dada2 feature-table
!qiime feature-table summarize \
  --i-table merged-dada2-filt10-raref-table-vole137-apo272.qza \
  --m-sample-metadata-file metadata-vole137-apo272.txt \
  --o-visualization merged-dada2-filt10-raref-table-vole137-apo272.qzv \
  
#examine merged representative sequences
!qiime feature-table tabulate-seqs \
 --i-data merged-dada2-rep-seqs-vole137-apo272.qza \
 --o-visualization merged-dada2-rep-seqs-vole137-apo272.qzv

In [ ]:
!qiime diversity core-metrics \
  --i-table merged-dada2-filt10-raref-table-vole137-apo272.qza \
  --p-sampling-depth 10553 \
  --m-metadata-file metadata-vole137-apo272.txt \
  --output-dir core-metrics-merged-filt10-vole137-apo272-results/

Repeat the same merging procedure, but use the 'chernobyl142.qza' data set instead of all the data. This is because this approach above shows that the strongest signal comes from the difference between species (from Europe vs. from Japan), and it is difficult to make any inference on the effects of the treatment (e.g. radiation exposure). Note that the '-vole137' was added to the 'core-metrics-filt10-vole137-results/rarefied_table.qza' file name for clarity and unique files name list (e.g. resulted in 'rarefied_table-vole137.qza').

In [ ]:
#merge tables
!qiime feature-table merge \
  --i-tables core-metrics-filt10-vole137-results/rarefied_table-vole137.qza \
  --i-tables apo272-original-files/rarefied_table_chernobyl142.qza \
  --o-merged-table cez-merged-dada2-filt10-raref-table-vole137-apo142.qza

In [ ]:
#summarize
!qiime feature-table summarize \
  --i-table cez-merged-dada2-filt10-raref-table-vole137-apo142.qza \
  --m-sample-metadata-file metadata-vole137-apo272.txt \
  --o-visualization cez-merged-dada2-filt10-raref-table-vole137-apo142.qzv \

In [ ]:
!qiime diversity core-metrics \
  --i-table cez-merged-dada2-filt10-raref-table-vole137-apo142.qza \
  --p-sampling-depth 10553 \
  --m-metadata-file metadata-vole137-apo272.txt \
  --output-dir core-metrics-merged-filt10-vole137-apo142-CEZ-results/

Note that despite the fact that voles and mice clearly characterised by species-specific gut microbiota profiles (likely because they share only few ASVs), all these species also characterised by significant treatment effects (note that only the non-phylogenetic beta diversity metrics were used for comparison).

In [ ]:
!qiime diversity adonis \
--i-distance-matrix core-metrics-merged-filt10-vole137-apo142-CEZ-results/bray_curtis_distance_matrix.qza \
--m-metadata-file metadata-vole137-apo272.txt \
--p-formula "HostSpecies+Treatment" \
--o-visualization core-metrics-merged-filt10-vole137-apo142-CEZ-results/bray-adonis-HostSpecies-Treatment.qzv

In [ ]:
!qiime diversity adonis \
--i-distance-matrix core-metrics-merged-filt10-vole137-apo142-CEZ-results/jaccard_distance_matrix.qza \
--m-metadata-file metadata-vole137-apo272.txt \
--p-formula "HostSpecies+Treatment" \
--o-visualization core-metrics-merged-filt10-vole137-apo142-CEZ-results/jaccard-adonis-HostSpecies-Treatment.qzv

## effects of frequency-based feature-table filtering

Examine the effects of frequency-based feature-table filtering (e.g. filtering of the low-abundance features; compare with vs. without 'filter10') on the alpha and beta diveristy estmates.

In [ ]:
#rarefy the original apo272 table to the same 10553 reads/sample
!qiime feature-table rarefy \
--i-table apo272-original-files/apo272-table-dada2.qza \
--p-sampling-depth 10553 \
--o-rarefied-table apo272-original-files/apo272-table-dada2-raref10553-orig.qza

#examine feature-table
!qiime feature-table summarize \
--i-table apo272-original-files/apo272-table-dada2-raref10553-orig.qza \
--o-visualization apo272-original-files/apo272-table-dada2-raref10553-orig.qzv

#compute non-phylogenetic alpha-diversity metrics
!qiime diversity alpha \
--i-table apo272-original-files/apo272-table-dada2-raref10553-orig.qza \
--p-metric 'observed_otus' \
--output-dir apo272-original-files/adiv-orig-apo272

Run the core-metrics-phylogenetic pipeline on this 'original' data in order to examine the effects of filtering also on beta diversity metrics (using Mantel tests).

In [ ]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny apo272-original-files/dada2-apo272/rooted-apo272-tree-dada2.qza \
  --i-table apo272-original-files/apo272-table-dada2-raref10553-orig.qza \
  --p-sampling-depth 10553 \
  --m-metadata-file apo272-original-files/apodemus_metadata181219_rev1ME.txt \
  --output-dir apo272-original-files/core-metrics-apo272-dada2-orig-results/

In [ ]:
#sanity check the 'rarefied' feature table generated in the core-metrics pipeline above
#examine feature-table
!qiime feature-table summarize \
--i-table apo272-original-files/core-metrics-apo272-dada2-orig-results/rarefied_table.qza \
--o-visualization apo272-original-files/core-metrics-apo272-dada2-orig-results/rarefied_table.qzv \

Run Mantel tests to examine consistency of the beta diversity results to the frequency-based feature-table filtering (e.g. 'filter10').

In [ ]:
!qiime diversity mantel \
--i-dm1 apo272-original-files/core-metrics-apo272-dada2-orig-results/bray_curtis_distance_matrix.qza \
--i-dm2 apo272-original-files/dada2-apo272/core-metrics-apo272-dada2-filtered10-results/bray_curtis_distance_matrix.qza \
--p-method spearman \
--p-permutations 999 \
--p-label1 'Original Distance Matrix' \
--p-label2 'Filtered Distance Matrix' \
--o-visualization apo272-original-files/bdiv-orig-apo272/bray-filt10test-mantel.qzv

In [ ]:
!qiime diversity mantel \
--i-dm1 apo272-original-files/core-metrics-apo272-dada2-orig-results/jaccard_distance_matrix.qza \
--i-dm2 apo272-original-files/dada2-apo272/core-metrics-apo272-dada2-filtered10-results/jaccard_distance_matrix.qza \
--p-method spearman \
--p-permutations 999 \
--p-label1 'Original Distance Matrix' \
--p-label2 'Filtered Distance Matrix' \
--o-visualization apo272-original-files/bdiv-orig-apo272/jaccard-filt10test-mantel.qzv

In [ ]:
!qiime diversity mantel \
--i-dm1 apo272-original-files/core-metrics-apo272-dada2-orig-results/unweighted_unifrac_distance_matrix.qza \
--i-dm2 apo272-original-files/dada2-apo272/core-metrics-apo272-dada2-filtered10-results/unweighted_unifrac_distance_matrix.qza \
--p-method spearman \
--p-permutations 999 \
--p-label1 'Original Distance Matrix' \
--p-label2 'Filtered Distance Matrix' \
--o-visualization apo272-original-files/bdiv-orig-apo272/unwUF-filt10test-mantel.qzv

In [ ]:
!qiime diversity mantel \
--i-dm1 apo272-original-files/core-metrics-apo272-dada2-orig-results/weighted_unifrac_distance_matrix.qza \
--i-dm2 apo272-original-files/dada2-apo272/core-metrics-apo272-dada2-filtered10-results/weighted_unifrac_distance_matrix.qza \
--p-method spearman \
--p-permutations 999 \
--p-label1 'Original Distance Matrix' \
--p-label2 'Filtered Distance Matrix' \
--o-visualization apo272-original-files/bdiv-orig-apo272/wUF-filt10test-mantel.qzv

## ds-fdr differential abundance testing

Run the [ds-fdr plugin](https://forum.qiime2.org/t/q2-dsfdr-community-tutorial/5559) within the qiime2 to identify differentially abundant features within the *Apodemus* data set. Note that the ds-fdr method can test differences between two groups only, read the tutorial notes to help results interpretation. Also note that prior to run the ds-fdr it first needs to be installed as a qiime2 dev plugin via pip install (see more details [here](https://forum.qiime2.org/t/q2-dsfdr-community-tutorial/5559)). This was run with the qiime2 version change to 2020.2.

The parameters for the number of permutations and discrete FDR correction level (e.g. alpha) were adopted from the settings used in the following [paper](https://msystems.asm.org/content/3/3/e00031-18.full).

In [ ]:
!qiime dsfdr permutation-fdr \
--i-table rarefied_table_chernobyl115_flavicollis.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column 'Treatment' \
--p-alpha 0.1 \
--p-permutations 10000 \
--o-visualization rarefied_table_chernobyl115_flavicollis_a01_p10k_dsfdr.qzv \
--verbose


!qiime dsfdr permutation-fdr \
--i-table rarefied_table_chernobyl27_sylvaticus.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column 'Treatment' \
--p-alpha 0.1 \
--p-permutations 10000 \
--o-visualization rarefied_table_chernobyl27_sylvaticus_a01_p10k_dsfdr.qzv \
--verbose


!qiime dsfdr permutation-fdr \
--i-table rarefied_table_fukushima77_speciosus_large.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column 'Treatment' \
--p-alpha 0.1 \
--p-permutations 10000 \
--o-visualization rarefied_table_fukushima77_speciosus_large_a01_p10k_dsfdr.qzv \
--verbose


!qiime dsfdr permutation-fdr \
--i-table rarefied_table_fukushima53_argenteus_small.qza \
--m-metadata-file apodemus_metadata181219.txt \
--m-metadata-column 'Treatment' \
--p-alpha 0.1 \
--p-permutations 10000 \
--o-visualization rarefied_table_fukushima53_argenteus_small_a01_p10k_dsfdr.qzv \
--verbose